# Cleaning Data and Visualization

This module marks a subtle change. Until now, you have been introduced to ideas and techniques that prepare you for real world situations. You are now going to start using these tools while learning how and when to use them in your work with data.

Before we get to the colorful world of graphing our data, we have to contend with the fact that real world data is messy. You will likely need to combine several data sources to get the data you want. The data from those sources will be incomplete. It will not likely be formatted in exactly the way you want to perform your analysis. It's for these reasons that most data scientists believe that about 80 percent of any project is spent getting the data into a form ready for analysis. Our graphs will only make sense after combing through our data and cleaning up the sore spots.

## Exploring `DataFrame` information

Once you have loaded your data into pandas, it will likely be in a `DataFrame`. However, if the dataset has 60,000 rows and 400 columns, how do you get a sense of what you're working with? Fortunately, pandas provides some convenient tools to provide a quick look at the overall information in a `DataFrame`.

To explore this functionality, you will import the Python `scikit-learn` library which contains example datasets for practice. We will use the "Iris plants" dataset which contains measurements performed on flowers belonging to the Iris species.

![](media/iris_organs.png)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
iris_df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])

### `DataFrame.info`
Let's take a look at this dataset:

In [ ]:
iris_df.info()

From this, you know that the *Iris* dataset has 150 entries across four columns. The data is stored as 64-bit floating-point numbers.

### `DataFrame.head` and `DataFrame.tail`
To get a quick snapshot of what the dataset looks like, you can check its first/last few rows using head()/tail() respectively:

In [ ]:
iris_df.head()

In [ ]:
iris_df.tail()

### Exercise:

By default, `DataFrame.head` returns the first five rows of a `DataFrame`. In the code cell below, can you figure out how to get it to show more?

In [ ]:
# Hint: Consult the documentation by using "iris_df.head?"

> **Takeaway:** The metadata in a DataFrame, or the first and last few values can get you an initial idea about the size, shape, and content of the data at hand.

## Dealing with missing data

> **Learning goal:** By the end of this subsection, you should know how to replace or remove null values from DataFrames.

Most of the time the datasets you want to use have missing values in them. How missing data is handled carries with it subtle tradeoffs that can affect your final analysis and real world outcomes.

Pandas handles missing values in two ways. The first is `NaN`, or **Not a Number**, which is used to indicate missing floating-point values. The second way of handling missing values is using the Python `None` keyword, which signals the lack of an object in general.

Here's a quick summary detailing the differences between `None` and `NaN`:

|   |`NaN` (NumPy.nan)|`None`|
|---|-----|------|
|dtype|'float'|General 'object'|
|Supports math?|Yes (any operation results in `NaN`)|No, not a numeric type|
|Supports aggregation?| Yes (results in `NaN`)|No, results in a crash|
|Upcasting?|casts any **numerical** array to a 'float' array|casts any array to an 'object' array|

**Note**: In other object-oriented programming languages (Java, C#/C++ etc), the `null` keyword is used instead to refer to the absence of an object. In the context of pandas/NumPy however, `null` encapsulates both cases of having either a `None` or a `NaN` as you will see in the following sections.

### Exercise:

In [ ]:
# What happens if you add np.nan to an array of integers?
example3 = np.array([1, 2, 3])
example3.dtype

> **Remember**: `NaN` is for missing floating-point values; there is no `NaN` equivalent for integers, strings, or Booleans.

### `NaN` and `None`: null values in pandas

Even though `NaN` and `None` can behave somewhat differently, pandas is built to handle them interchangeably. To see this, consider a `Series` of integers:

In [ ]:
int_series = pd.Series([1, 2, 3], dtype=int)
int_series

### Exercise:

In [ ]:
# Now set an element of int_series equal to None.
# How does that element show up in the series?
# What is the dtype of the series?


In the process of upcasting data types to establish data homogeneity in `Series` and `DataFrame`s, pandas will willingly switch missing values between `None` and `NaN`. Because of this design feature, it can be helpful to think of `None` and `NaN` as two flavors of "null" in pandas. Indeed, some of the core methods you will use to deal with missing values in pandas reflect this idea in their names:

- `isnull()`: Generates a Boolean mask indicating missing values
- `notnull()`: Complement of `isnull()`
- `dropna()`: Returns a filtered version of the data
- `fillna()`: Returns a copy of the data with missing values filled or imputed

These are important methods to master and get comfortable with, so let's go over them each in some depth.

### Detecting null values
Both `isnull()` and `notnull()` are your primary methods for detecting null data. Both return Boolean masks over your data.

In [ ]:
example3 = pd.Series([0, np.nan, '', None, 'NaN'])
example3

In [ ]:
example3.isnull()

Look closely at the output. Does any of it surprise you? While `0` is an arithmetic null, it's nevertheless a perfectly good integer and pandas treats it as such. `''` is a little more subtle. It is a string object and not a representation of null as far as pandas is concerned. `'NaN'` is also a string object that has no relation to the floating-point `NaN` value.

Now, let's turn this around and use these methods in a manner more like you will use them in practice. You can use Boolean masks directly as a ``Series`` or ``DataFrame`` index, which can be useful when working with isolated missing (or present) values.

In [ ]:
# Try running example3[example3.notnull()].
# Before you do so, what do you expect to see?


**Key takeaway**: Both the `isnull()` and `notnull()` methods produce similar results when you use them in `DataFrame`s: they show the results and the index of those results, which will help you enormously as you wrestle with your data.

### Dropping null values

Beyond identifying missing values, pandas provides a convenient means to remove null values from `Series` and `DataFrame`s. Particularly on large datasets, it is often more advisable to remove missing [NA] values from your analysis than deal with them in other ways. To see this in action, return to `example3`:

In [ ]:
example3 = example3.dropna()
example3

Note that this should look like your output from `example3[example3.notnull()]`. The difference is that, rather than indexing on the masked values, `dropna` has removed those missing values from the `Series` `example3`.

Because `DataFrame`s have two dimensions, they afford more options for dropping data.

In [ ]:
example4 = pd.DataFrame([[1,      np.nan, 7], 
                         [2,      5,      8], 
                         [np.nan, 6,      9]])
example4.dtypes

(Did you notice that pandas upcast two of the columns to floats to accommodate the `NaN`s?)

You cannot drop a single value from a `DataFrame`, so you have to drop full rows or columns. In data science columns generally represent variables and rows represent observations, you are more likely to drop rows of data; the default setting for `dropna()` is to drop all rows that contain null values:

In [ ]:
# By the way, dropna() does not directly modify example4 but rather returns a shallow copy
example4.dropna()

If necessary, you can remove columns with NA values by assigning `'columns'` to the keyword argument `axis`:

In [ ]:
example4.dropna(axis='columns')

Notice that this can drop a lot of data that you might want to keep, particularly in smaller datasets. What if you only want to drop rows or columns that contain several or even all null values? You specify those settings in `dropna` with the `how` and `thresh` parameters. Check out what these parameters do by reading the documentation:

In [ ]:
# Omit the parenthesis and add a question mark '?' to view info about a function
# If you get a warning about output size limit, just follow the link it gives you to get the full text
# Note that this only works on Jupyter Notebook code cells
example4.dropna?

In [ ]:
# Only drop columns that are composed entirely of nulls

example4[3] = np.nan
print("Before: ")
display(example4) #display() is just a nicer version of print() intended for Notebooks
print("After: ")
# Sometimes you will see 0 or 1 passed into the axis argument, 0 means rows and 1 means columns
example4.dropna(axis=1, how='all')

The `thresh` parameter gives you finer-grained control: you set the number of *non-null* values that a row or column needs to have to be kept.

### Exercise:

In [ ]:
# Consult the documentation for dropna(),
# Keep columns that have a minimum of 2 valid values, drop the rest
# Hint: You need to use the axis and thresh parameters

dirtyFrame = pd.DataFrame([['Title #0',      np.nan, 7], 
                         ['Title #1',      np.nan,      8], 
                         [None, 6,      9]])
display(dirtyFrame)
# Add your code after this comment!

### Filling null values

Depending on your dataset, it can sometimes make more sense to fill null values with valid ones rather than drop them. Because this is such a common task in data science, pandas provides `fillna`, which returns a copy of the `Series` or `DataFrame` with the missing values replaced with one of your choosing. Let's create another example `Series` to see how this works in practice.

In [ ]:
example5 = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
example5

You can fill the null entries with a single value, such as `0`:

In [ ]:
example5.fillna(0)

You can **forward-fill** null values, which is to use the last valid value to fill a null:

In [ ]:
example5.fillna(method='ffill')

You can also **back-fill** to propagate the next valid value backward to fill a null:

In [ ]:
example5.fillna(method='bfill')

As you might guess, this works the same with `DataFrame`s, but you can also specify an `axis` along which to fill null values:

In [ ]:
example4

In [ ]:
example4.fillna(method='ffill', axis='columns')

Notice that when a previous value is not available for forward-filling, the null value remains.

When the `axis` parameter is specified as `'columns'`, `fillna()` checks the cells along a column for nulls. If it encounters a null it retrieves the previous value from the column that came before it.

You can be creative about how you use `fillna`. For example, let's look at `example4` again, but this time fill the missing values with the average of all the values in the `DataFrame`:

In [ ]:
example4.fillna(example4.mean())

Notice that column 3 is still valueless: the default direction is to fill values row-wise.

> **Takeaway:** There are multiple ways to deal with missing values in your datasets. The specific strategy you use (removing them, replacing them, or even how you replace them) should be dictated by the particulars of that data. You will develop a better sense of how to deal with missing values the more you handle and interact with datasets.

## Removing duplicate data

In addition to missing data, you will often encounter duplicated data in real world datasets. Fortunately, pandas provides an easy means of detecting and removing duplicate entries.

### Identifying duplicates: `duplicated`

You can easily spot duplicate values using the `duplicated` method in pandas, which returns a Boolean mask indicating whether an entry in a `DataFrame` is a duplicate of an earlier one. Let's create another example `DataFrame` to see this in action.

In [ ]:
example6 = pd.DataFrame({'letters': ['A','B'] * 2 + ['B'],
                         'numbers': [1, 2, 1, 3, 3]})
example6

In [ ]:
example6.duplicated()

### Dropping duplicates: `drop_duplicates`
`drop_duplicates` simply returns a copy of the data for which all the `duplicated` values are `False`:

In [ ]:
example6.drop_duplicates()

Both `duplicated` and `drop_duplicates` default to consider all columns but you can specify that they examine only a subset of columns in your `DataFrame`:

In [ ]:
example6.drop_duplicates(['letters'])

## Combining datasets: merge and join

> **Learning goal:** By the end of this subsection, you should have a general knowledge of the various ways to combine `DataFrame`s.

Your most interesting analyses often comes from data melded together from more than one source. Pandas provides several methods of merging and joining datasets to make this necessary job easier:
 - **`pandas.merge`** connects rows in `DataFrame`s based on one or more keys.
 - **`pandas.concat`** concatenates or “stacks” together objects along an axis.
 - The **`combine_first`** instance method enables you to splice together overlapping data to fill in missing values in one object with values from another.

Let's examine merging data first, because it will be the most familiar to anyone already familiar with SQL or other relational databases.

### Categories of joins

`merge` carries out several types of joins: *one-to-one*, *many-to-one*, and *many-to-many*. You use the same basic function call to implement all of them and you will examine all three (because you will need all three at some point in your data-delving, depending on the data). You will start with one-to-one joins because they are generally the simplest.

#### One-to-one joins

Consider combining two `DataFrame`s that contain different information about the same employees in a company:

In [ ]:
df1 = pd.DataFrame({'employee': ['Amin', 'Amel', 'Maissa', 'Yassine'],
                    'group': ['R&D', 'Marketing', 'Marketing', 'HR']})
df1

In [ ]:
df2 = pd.DataFrame({'employee': ['Amin', 'Amel', 'Maissa', 'Yassine'],
                    'hire_date': [2008, 2012, 2017, 2018]})
df2

Combine this information into a single `DataFrame` using the `merge` function:

In [ ]:
df3 = pd.merge(df1, df2)
df3

Pandas joined on the `employee` column because it was the only column common to both `df1` and `df2` (note that the original indices of `df1` and `df2` were discarded by `merge`; this is generally the case with merges unless you conduct them by index, which you will learn later).

#### Many-to-one joins

A many-to-one join is like a one-to-one join except that one of the two key columns contains duplicate entries. The `DataFrame` resulting from such a join will preserve those duplicate entries as appropriate:

In [ ]:
df4 = pd.DataFrame({'group': ['R&D', 'Marketing', 'HR'],
                    'supervisor': ['Dhia', 'Chiraz', 'Baha']})
df4

In [ ]:
pd.merge(df3, df4)

The resulting `DataFrame` has an additional column for `supervisor`; that column has an extra occurrence of 'Chiraz' that did not occur in `df4` because more than one employee in the merged `DataFrame` works in the 'Marketing' group.

Note that you didn’t specify which column to join. When you don't specify that information, `merge` uses the overlapping column names as the keys. However, that can be ambiguous; several columns might meet that condition. For that reason, it is a good practice to explicitly specify on which key to join. You can do this with the `on` parameter:

In [ ]:
pd.merge(df3, df4, on='group')

#### Many-to-many joins
What happens if the key columns in both of the `DataFrame`s you are joining contain duplicates? That gives you a many-to-many join:

In [ ]:
df5 = pd.DataFrame({'group': ['R&D', 'R&D',
                              'Marketing', 'Marketing', 'HR', 'HR'],
                    'core_skills': ['coding', 'soldering', 'writing', 'communication',
                               'spreadsheets', 'organization']})
df5

In [ ]:
pd.merge(df1, df5, on='group')

Again, to avoid ambiguity as to which column to join on, it is a good idea to explicitly tell `merge` which one to use with the `on` parameter.

#### `left_on` and `right_on` keywords
What if you need to merge two datasets with no shared column names? For example, what if you are using a dataset in which the employee name is labeled as 'name' rather than 'employee'? In such cases, you will use the `left_on` and `right_on` keywords to specify the column names on which to join:

In [ ]:
df6 = pd.DataFrame({'name': ['Amin', 'Amel', 'Maissa', 'Yassine'],
                    'salary': [70000, 80000, 120000, 90000]})
df6

In [ ]:
group_salary = pd.merge(df1, df6, left_on="employee", right_on="name")
group_salary

### Exercise:

In [ ]:
# Using the documentation, can you figure out how to use .drop() to get rid of the 'name' column?
# Hint: You will need to supply two parameters to .drop().
group_salary.drop?

You can also use the `join` method for `DataFrame`s, which merges on indices by default. To use a different column as an index for a `DataFrame`, you can use `set_index()`:

In [ ]:
df1a = df1.set_index('employee')
display(df1a)
df2a = df2.set_index('employee')
display(df2a)

df1a.join(df2a)

You can also mix and match `left_index`/`right_index` with `right_on`/`left_on`:

#### Set arithmetic for joins

Let's return to many-to-many joins for a moment. A consideration that is unique to them is the *arithmetic* of the join--specifically, the set arithmetic you use for the join. To illustrate what this, let's restructure an old example `DataFrame`:

In [ ]:
df5 = pd.DataFrame({'group': ['Engineering', 'Marketing', 'Sales'],
                    'core_skills': ['math', 'writing', 'communication']})
df5

In [ ]:
pd.merge(df1, df5, on='group')

Notice that after you have restructured `df5` and then re-run the merge with `df1`, you have only two entries in the result. This is because you merged on `group` and 'Marketing' was the only entry that appeared in the `group` column of both `DataFrame`s.

In effect, what you have gotten is the *intersection* of both `DataFrame`s. This is known as the inner join in the database world and it is the default setting for `merge`, although you can certainly specify it:

In [ ]:
pd.merge(df1, df5, on='group', how='inner')

The complement of the inner join is the outer join, which returns the *union* of the two `DataFrame`s.

### Exercise:

In [ ]:
# The keyword for performing an outer join is how='outer'. How would you code it?
# What do you expect the output of an outer join of df1 and df5 to be?


Notice in your resulting `DataFrame` that not every row in `df1` and `df5` had a value that corresponds to the union of the key values (the 'group' column). Pandas fills in these missing values with `NaN`s.

Inner and outer joins are not your only options. A *left join* returns all the rows in the first (left-side) `DataFrame` supplied to `merge` along with rows from the other `DataFrame` that match the left-side key values (and `NaNs` rows with respective values):

In [ ]:
pd.merge(df1, df5, how='left')

### Exercise:

In [ ]:
# Now run the right merge between df1 and df5.
# What do you expect to see?


#### `suffixes` keyword: dealing with conflicting column names
Because you can join datasets, you will eventually join two with conflicting column names. Let's look at another example to see what this means:

In [ ]:
df7 = pd.DataFrame({'name': ['Gary', 'Stu', 'Mary', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df7

In [ ]:
df8 = pd.DataFrame({'name': ['Gary', 'Stu', 'Mary', 'Sue'],
                    'rank': [3, 1, 4, 2]})
df8

In [ ]:
pd.merge(df7, df8, on='name')

Each column name in a `DataFrame` must be unique, so in cases where two joined `DataFrame`s share column names (aside from the column serving as the key), the `merge` function automatically appends the suffix `_x` or `_y` to the conflicting column names to make them unique. In cases where it is best to control your column names, you can specify a custom suffix for `merge` to append through the `suffixes` keyword:

In [ ]:
pd.merge(df7, df8, on='name', suffixes=['_left', '_right'])

Note that these suffixes work if there are multiple conflicting columns.

### Concatenation in NumPy
Concatenation in pandas is built off of the concatenation functionality for NumPy arrays. Here is what NumPy concatenation looks like:
 - For one-dimensional arrays:

In [ ]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

 - For two-dimensional arrays:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

Notice that the `axis=1` parameter makes the concatenation occur along columns rather than rows. Concatenation in pandas looks similar to this.

### Concatenation in pandas

Pandas has a function, `pd.concat()`, that can be used for a simple concatenation of `Series` or `DataFrame` objects in a similar manner to `np.concatenate()` with ndarrays.

In [ ]:
ser1 = pd.Series(['a', 'b', 'c'], index=[1, 2, 3])
ser2 = pd.Series(['d', 'e', 'f'], index=[4, 5, 6])
pd.concat([ser1, ser2])

It also concatenates higher-dimensional objects, such as ``DataFrame``s:

In [ ]:
df9 = pd.DataFrame({'A': ['a', 'c'],
                    'B': ['b', 'd']})
df9

In [ ]:
pd.concat([df9, df9])

Notice that `pd.concat` has preserved the indexing even though that means that it has been duplicated. You can have the results re-indexed--and avoid potential confusion later--like so:

In [ ]:
pd.concat([df9, df9], ignore_index=True)

By default, `pd.concat` concatenates row-wise within the `DataFrame` (that is, `axis=0` by default). You can specify the axis along which to concatenate:

In [ ]:
pd.concat([df9, df9], axis=1)

Note that while pandas will display this without error, you will get an error message if you try to assign this result as a new `DataFrame`. Column names in `DataFrame`s must be unique.

### Concatenation with joins
Just as you did with merge above, you can use inner and outer joins when concatenating `DataFrame`s with different sets of column names.

In [ ]:
df10 = pd.DataFrame({'A': ['a', 'd'],
                     'B': ['b', 'e'],
                     'C': ['c', 'f']})
df10

In [ ]:
df11 = pd.DataFrame({'B': ['u', 'x'],
                     'C': ['v', 'y'],
                     'D': ['w', 'z']})
df11

In [ ]:
pd.concat([df10, df11])

As you saw earlier, the default join for this is an outer join, and entries for which no data is available are filled with `NaN` values. You can also do an inner join:

In [ ]:
pd.concat([df10, df11], join='inner')

#### `append()`

Because direct array concatenation is so common, ``Series`` and ``DataFrame`` objects have an ``append`` method that can accomplish the same thing in fewer keystrokes. For example, rather than calling ``pd.concat([df9, df9])``, you can simply call ``df9.append(df9)``:

In [ ]:
df9.append(df9)

**Important point**: Unlike the `append()` and `extend()` methods of Python lists, the `append()` method in pandas does not modify the original object. Instead, it creates a new object with the combined data.

> **Takeaway:** A large part of the value you can provide as a data scientist comes from connecting multiple, often disparate, datasets to find new insights. Learning how to join and merge data is an essential part of your skill set.

## Exploratory statistics and visualization

> **Learning goal:** By the end of this subsection, you should be familiar with some of the ways to visually explore the data stored in `DataFrame`s.

Often when probing a new dataset, it is invaluable to get high-level information about what the dataset holds. Earlier in this section, you used methods, such as `DataFrame.info`, `DataFrame.head`, and `DataFrame.tail` to examine some aspects of a `DataFrame`. While these methods are critical, they are often insufficient to get enough information to know how to approach a new dataset. This is where exploratory statistics and visualizations for datasets come in.

To see what this means in terms of gaining exploratory insight (both visually and numerically), let's dig into the Pizza Sales dataset. It is given to you as a CSV file that can easily be read and converted into a pandas DataFrame, pandas also supports Excel spreadsheets, JSON files, SQL tables... 

In [ ]:
df = pd.read_csv('Data/pizza_dataset.csv')
df.head()

This dataset contains information concerning the sales of a pizzeria during the month of March 2015, it is organized as follows:
 - **pizza_id**: Unique identifier for each pizza that has been ordered.
 - **order_id**: Unique identifier for each order, an order can contain multiple pizzas.
 - **pizza_name**: The kind of pizza that has been ordered.
 - **quantity**: Number of pies, multiple pizzas of the same name and same order have the same `pizza_id`.
 - **order_date**: Date of purchase in DD/MM/YYYY format.
 - **order_time**: Time of purchase in HH:MM format.
 - **unit_price**: Price of one pizza pie.
 - **total_price**: Price of one pizza of this type multiplied by the quantity ordered.
 - **pizza_size**: Pizza size: `S`mall, `M`edium, `L`arge, E`x`tra `L`arge or E`x`tra E`x`tra `L`arge.
 - **pizza_category**: Categories that group many pizza types into the same theme: Chicken, Veggie, Supreme, Classic.

One of the first methods you can use to understand this dataset is `DataFrame.shape`:

In [ ]:
df.shape

The dataset has 4186 rows and 11 columns.

You can examine specific descriptive statistics for numerical columns:

In [ ]:
df['unit_price'].mean()

In [ ]:
# Why is it better to look at the median rather than the average?
df['quantity'].median()

### Exercise:

In [ ]:
# We want to find the maximum value of money spent on a single pizza type,
# but it seems that the total_price column is all nulls! 
# Fix it by filling with the correct values before trying to find the max.

df['total_price'].max()

Other information that you will want to see is the relationship between different columns. You do this with the `DataFrame.groupby` method. For example, you could examine the average `unit_price` for each value of `pizza_category`:

In [ ]:
df.groupby(['pizza_category'])['unit_price'].mean()

### Exercise:

In [ ]:
# Now try to find the median value for 'unit_price' for each value of 'pizza_size'.

Data may not always arrive to us in a convenient form, for example the pizza sizes were recorded in terms of the typical designatons you'd see at pizzerias (M for medium, L for Large...) but let's say we wanted a numerical interpretation instead, we can rectify this by replacing those strings with the diameter in centimeteres:

In [ ]:
for i in range(0, len(df.index)):
    catSize = df['pizza_size'][i]
    if (catSize == 'S'):
        df['pizza_size'][i] = 25
    elif (catSize == 'M'):
        df['pizza_size'][i] = 30
    elif (catSize == 'L'):
        df['pizza_size'][i] = 35
    elif (catSize == 'XL'):
        df['pizza_size'][i] = 40
    elif (catSize == 'XXL'):
        df['pizza_size'][i] = 45
# Cast the size series into int so that we can do math on it         
df['pizza_size'] = df['pizza_size'].astype(int)
df

We will also create a new `Hour` column based on `order_time`:

In [ ]:
df['order_time']=df['order_time'].astype('string')
df[['Hour','Minute']]= df['order_time'].str.split(":",expand=True)
df['Hour'] = df['Hour'].astype(int)
df['Hour']

Python provides a special type for handling date and time information, let's cast the `order_date` column into `datetime64`:

In [ ]:
df['order_date'] = pd.to_datetime(df.order_date, format="%d/%m/%Y")
df['order_date']

It is often valuable to know how many unique values a column has in it with the `nunique` method:

In [ ]:
df['pizza_name'].nunique()

Complementary to that, you will likely want to know what those unique values are, which is where the `unique` method helps:

In [ ]:
df['pizza_name'].unique()

You can use the `value_counts` method to see how many of each unique value there are in a column:

In [ ]:
df['pizza_name'].value_counts()

Or you can easily plot a bar graph to visually see the breakdown:

In [ ]:
# pandas relies on matplotlib by default to plot its graphs

df['pizza_name'].value_counts().plot(kind='bar')

Let's pull back to the dataset as a whole for a moment. Two major things that you will look for in almost any dataset are trends and relationships. A typical relationship between variables to explore is the Pearson correlation coefficient, or the extent to which two variables are linearly related.

![](media/pearson_coefficient.png)

The `corr` method will show this in table format for all the columns in a `DataFrame`:

In [ ]:
df.corr(method='pearson')

Suppose you wanted to look at the correlations between all the columns and only one variable. Let's examine the correlation between all other variables and the total price. You will do this by accessing the column by index:

In [ ]:
corr = df.corr(method='pearson')
corr_with_totalprice = corr.loc["total_price"]
corr_with_totalprice[corr_with_totalprice.argsort()]

### heatmap

Graphically, you can see the correlations using a heatmap from the Seaborn library:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#The argument is a color palette that maps:
# weak corr -> strong corr into light -> dark
sns.heatmap(df.corr(),cmap=sns.cubehelix_palette(20, light=0.95, dark=0.15))

### histplot

Histograms are another valuable tool for investigating your data. For example, what is the overall distribution of orders across every day of the month?

In [ ]:
#(width, height) adjustment
sns.set(rc={"figure.figsize":(10, 4)})
#change color palette
sns.set_palette("bright")
#print labels at a diagonal to fit them
plt.xticks(rotation = 25)


sns.histplot(df['order_date'])

Histograms group data into intervals or "bins", we can set the binwidth argument to get more control over how wide the bars are:

In [ ]:
# Usually we just pass the Series directly, 
# but it has been explicitly assigned to the y-axis to fit the long text
sns.histplot(y = df['order_date'], binwidth = 4)

We can enable the `kde` parameter to obtain a Kernel Density Estimation curve, it serves as a continuous envelope for our histogram: 

In [ ]:
plt.xticks(rotation = 25)
sns.histplot(df['order_date'], binwidth = 4, kde=True)

### jointplot

Another commonly-used plot is the Seaborn jointplot, which combines histograms for two columns along with a scatterplot:

In [ ]:
sns.jointplot(data = df, x = 'order_id', y = 'total_price', kind='scatter')

Unfortunately, many of the dots print over each other. You can address this by adding some alpha blending, a figure that sets the transparency for the dots so that concentrations of them drawing over one another will be apparent:

In [ ]:
sns.jointplot(data = df, x = 'order_id', y = 'total_price', kind='scatter', alpha=0.3)

### kdeplot

Another way to see patterns in your data is with a two-dimensional KDE plot. Darker colors here represent a higher concentration of data points:

In [ ]:
sns.kdeplot(data = df, x = 'order_id', y = 'total_price', fill=True)

Note that while the KDE plot is very good at showing concentrations of data points, finer structures like linear relationships (such as the clear relationship between the unit price and the total price) are lost in the KDE plot.

### Linear regression and pairplot

Linear regression is an algorithm that attempts to show a linear relationship between two variables, it can be used to interpolate values between gaps in the x-axis and also as a rough indicator of where the data is headed when we extrapolate to x-axis values beyond the range we have at hand.

![](media/linear_regression_example.png)

In linear regression, the observations (red) are assumed to be the result of random deviations (green) from an underlying relationship (blue) between a dependent variable (y) and an independent variable (x).

The pairplot in Seaborn allows you to see scatterplots and histograms for several columns in one table. Below, some of the keywords have been altered to produce a more sophisticated and easier to read pairplot that incorporates both alpha blending and linear regression lines for the scatterplots.

In [ ]:
sns.pairplot(df[['pizza_size', 'num_of_ingredients', 'total_price']], kind="reg", plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.1}})

### boxplot (Box & Whisker)

Box plots are used to show the spread and distribution of data points across "quartiles". Quartiles divide our data series into quarters, therefore the three quartiles are defined as:

* Q1: Lower quartile, 25% of the data is below this point.
* Q2: Middle quartile (median), 50% of the data is below this point.
* Q3: Upper quartile, 75% of the data is below this point.

![](media/box_plot.png)

It is common to exclude outliers (typically >150% deviation from the mean), they are often still shown as single data points outside the box-whisker structure.

Let's see the distribution of pizza orders depending on the hour of day:

In [ ]:
sns.boxplot(x=df['Hour'])

### Pie charts

Pie charts round up our data into a 360° circle, the circle is then sliced up according to categories making it excellent for visualizing value counts of non-numerical data. Seaborn does not offer a pie chart utility at the moment, so we'll have to manually tweak Matplotlib to get what we want:

In [ ]:
categoryCounts = df['pizza_category'].value_counts()
categoryNames = categoryCounts.index

# pass the data, display percentages with 2 places after the decimal point
plt.pie(x = categoryCounts, labels = categoryNames, autopct='%.2f%%')
plt.show()

Visualization is the start of the really cool, fun part of data science. So play around with these visualization tools and see what you can learn from the data!

> **Takeaway:** An old joke goes, “What do data scientists see when they look at a dataset? A bunch of numbers.” There is more than a little truth in that joke. Visualization is often the key to finding patterns and correlations in your data. While visualization cannot deliver precise results, it can point you in the right direction to ask better questions and efficiently find value in the data.